In [2]:
from output_reporter import OutputReporter
example_path = 'wiki_50/qa_BM25_wikipedia_50_5000_naturalQuestions-dev-clean_1000.json'
reporter = OutputReporter(example_path)
print(reporter.items[0])

{'question_id': 'https://en.wikipedia.org//w/index.php?title=History_of_the_Ottoman_Empire_during_World_War_I&amp;oldid=821344729', 'question': 'when did the ottoman empire surrender in ww1', 'pred_answers': [], 'gold_answers': ['1918', '30 October 1918'], 'bm25_ranks': [0, 515, 517, 518, 4614, 520, 521, 4111, 3603, 4631, 1049, 2585, 539, 4121, 544, 48, 1074, 1075, 569, 4156, 4668, 1598, 2114, 3651, 2118, 76, 2126, 3664, 1627, 2146, 4209, 115, 1140, 1141, 631, 1148, 2694, 4235, 652, 1171, 3221, 4248, 3743, 673, 1697, 1699, 164, 173, 174, 3246, 4283, 2748, 3261, 3776, 3777, 4295, 1229, 1741, 4815, 2770, 726, 1756, 736, 1765, 744, 2283, 1780, 2811, 1276, 4860, 254, 3848, 265, 1291, 780, 3340, 4363, 4877, 272, 2839, 798, 3870, 1824, 293, 4901, 2855, 808, 809, 810, 1835, 4392, 301, 4394, 1845, 823, 1850, 1851, 1864, 1865, 1873, 338, 4949, 359, 871, 875, 876, 1388, 3952, 4977, 1399, 1911, 3447, 4986, 379, 2428, 1933, 919, 2455, 1438, 2974, 4513, 2985, 426, 432, 2483, 2995, 3514, 444, 2495, 

In [7]:
import plotly.express as px

# df = px.data.gapminder().query("country=='Canada'")
# df = [(1,1), (2,2)]
fig = px.line(df, x="year", y="lifeExp", title='Life expectancy in Canada')
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of [0, 1] but received: year

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame([(1,1), (2,2)])